In [11]:
import os
from onai.ml.tools.spark import get_spark
spark = get_spark()

In [12]:
def spark_load(spark,p,extra_options = None,persist = False,temp_view_name= None):
    
    fmt, remained_p = p.split("://", 1)
    reader = spark.read
    if extra_options:
        reader = reader.options(**extra_options)
    if fmt in SUPPORTED_FMT_LIST:
        reader = reader.format(fmt)
        p = remained_p
    res_df = reader.load(p)
    if persist:
        res_df = res_df.persist()
    if temp_view_name is not None:
        res_df.createOrReplaceTempView(temp_view_name)
        
    return res_df

SUPPORTED_FMT_LIST = {'orc', 'parquet', 'csv'}

In [13]:
business_dir = 's3://one-lake-prod/ml-snapshots/20220116/business'

spark_load(spark, os.path.join(business_dir, 'financial_data'), temp_view_name='fd')
spark_load(spark, os.path.join(business_dir, 'company_data_denormalized'), temp_view_name='cdd')
spark_load(spark, os.path.join(business_dir, 'exchange_rate'), temp_view_name='er')
spark_load(spark, os.path.join(business_dir, 'company_data'), temp_view_name='cd')

spark_load(spark, 'csv://' + os.path.join(business_dir, 'dime_map', 'dime.csv'), 
           dict(header='True', inferSchema='True', delimiter=','), temp_view_name='dm')

DataFrame[oneid: string, mnemonic: string, provider_name: string, provider_data_item_id: string, name: string, accounting_concept_code: string, is_symbol: boolean, is_monetary: boolean, is_fiscal: boolean, is_array: boolean, value_type: string, description: string]

In [2]:
# query = "select entity_id from cdd limit {}".format(100)
# df = spark.sql(query)
# pdf = df.toPandas()
# pdf.head()

In [3]:
# query = """select cdd.entity_id,cdd.primary_naics_node,dm.mnemonic,event_date, fd.data_value from fd 
#         join dm on dm.oneid = fd.data_item_id and dm.provider_name = 'CapitalIQ'
#         join cdd on cdd.entity_id = fd.entity_id
#         where dm.mnemonic in ('TOTAL_REVENUE','EBITDA','NET_DEBT_EBITDA','TOTAL_DEBT_EQUITY')
#         and fd.period_type = 'ANNUAL'
#         and cdd.country_of_incorporation_iso = 'USA'
#         and cdd.primary_naics_node = {}
#         order by entity_id, fd.event_date desc
#         """.format('2111')

# df = spark.sql(query)
# sector_data = df.toPandas()
# sector_data.head()

In [2]:
string = """111140
111150
111160
111191
111199
111211
111219
111310
111320
111331
111332
111333
111334
111335
111336
111339
111411
111419
111421
111422
111910
111920
111930
111940
111991
111992
111998
112111
112112
112120
112210
112310
112320
112330
112340
112390
112410
112420
112511
112512
112519
112910
112920
112930
112990
113110
113210
113310
114111
114112
114119
114210
115111
115112
115113
115114
115115
115116
115210
115310
211120
211130
212111
212112
212113
212210
212221
212222
212230
212291
212299
212311
212312
212313
212319
212321
212322
212324
212325
212391
212392
212393
212399
213111
213112
213113
213114
213115
221111
221112
221113
221114
221115
221116
221117
221118
221121
221122
221210
221310
221320
221330
236115
236116
236117
236118
236210
236220
237110
237120
237130
237210
237310
237990
238110
238120
238130
238140
238150
238160
238170
238190
238210
238220
238290
238310
238320
238330
238340
238350
238390
238910
238990
311111
311119
311211
311212
311213
311221
311224
311225
311230
311313
311314
311340
311351
311352
311411
311412
311421
311422
311423
311511
311512
311513
311514
311520
311611
311612
311613
311615
311710
311811
311812
311813
311821
311824
311830
311911
311919
311920
311930
311941
311942
311991
311999
312111
312112
312113
312120
312130
312140
312230
313110
313210
313220
313230
313240
313310
313320
314110
314120
314910
314994
314999
315110
315190
315210
315220
315240
315280
315990
316110
316210
316992
316998
321113
321114
321211
321212
321213
321214
321219
321911
321912
321918
321920
321991
321992
321999
322110
322121
322122
322130
322211
322212
322219
322220
322230
322291
322299
323111
323113
323117
323120
324110
324121
324122
324191
324199
325110
325120
325130
325180
325193
325194
325199
325211
325212
325220
325311
325312
325314
325320
325411
325412
325413
325414
325510
325520
325611
325612
325613
325620
325910
325920
325991
325992
325998
326111
326112
326113
326121
326122
326130
326140
326150
326160
326191
326199
326211
326212
326220
326291
326299
327110
327120
327211
327212
327213
327215
327310
327320
327331
327332
327390
327410
327420
327910
327991
327992
327993
327999
331110
331210
331221
331222
331313
331314
331315
331318
331410
331420
331491
331492
331511
331512
331513
331523
331524
331529
332111
332112
332114
332117
332119
332215
332216
332311
332312
332313
332321
332322
332323
332410
332420
332431
332439
332510
332613
332618
332710
332721
332722
332811
332812
332813
332911
332912
332913
332919
332991
332992
332993
332994
332996
332999
333111
333112
333120
333131
333132
333241
333242
333243
333244
333249
333314
333316
333318
333413
333414
333415
333511
333514
333515
333517
333519
333611
333612
333613
333618
333912
333914
333921
333922
333923
333924
333991
333992
333993
333994
333995
333996
333997
333999
334111
334112
334118
334210
334220
334290
334310
334412
334413
334416
334417
334418
334419
334510
334511
334512
334513
334514
334515
334516
334517
334519
334613
334614
335110
335121
335122
335129
335210
335220
335311
335312
335313
335314
335911
335912
335921
335929
335931
335932
335991
335999
336111
336112
336120
336211
336212
336213
336214
336310
336320
336330
336340
336350
336360
336370
336390
336411
336412
336413
336414
336415
336419
336510
336611
336612
336991
336992
336999
337110
337121
337122
337124
337125
337127
337211
337212
337214
337215
337910
337920
339112
339113
339114
339115
339116
339910
339920
339930
339940
339950
339991
339992
339993
339994
339995
339999
423110
423120
423130
423140
423210
423220
423310
423320
423330
423390
423410
423420
423430
423440
423450
423460
423490
423510
423520
423610
423620
423690
423710
423720
423730
423740
423810
423820
423830
423840
423850
423860
423910
423920
423930
423940
423990
424110
424120
424130
424210
424310
424320
424330
424340
424410
424420
424430
424440
424450
424460
424470
424480
424490
424510
424520
424590
424610
424690
424710
424720
424810
424820
424910
424920
424930
424940
424950
424990
425110
425120
441110
441120
441210
441222
441228
441310
441320
442110
442210
442291
442299
443141
443142
444110
444120
444130
444190
444210
444220
445110
445120
445210
445220
445230
445291
445292
445299
445310
446110
446120
446130
446191
446199
447110
447190
448110
448120
448130
448140
448150
448190
448210
448310
448320
451110
451120
451130
451140
451211
451212
452210
452311
452319
453110
453210
453220
453310
453910
453920
453930
453991
453998
454110
454210
454310
454390
481111
481112
481211
481212
481219
482111
482112
483111
483112
483113
483114
483211
483212
484110
484121
484122
484210
484220
484230
485111
485112
485113
485119
485210
485310
485320
485410
485510
485991
485999
486110
486210
486910
486990
487110
487210
487990
488111
488119
488190
488210
488310
488320
488330
488390
488410
488490
488510
488991
488999
491110
492110
492210
493110
493120
493130
493190
511110
511120
511130
511140
511191
511199
511210
512110
512120
512131
512132
512191
512199
512230
512240
512250
512290
515111
515112
515120
515210
517311
517312
517410
517911
517919
518210
519110
519120
519130
519190
521110
522110
522120
522130
522190
522210
522220
522291
522292
522293
522294
522298
522310
522320
522390
523110
523120
523130
523140
523210
523910
523920
523930
523991
523999
524113
524114
524126
524127
524128
524130
524210
524291
524292
524298
525110
525120
525190
525910
525920
525990
531110
531120
531130
531190
531210
531311
531312
531320
531390
532111
532112
532120
532210
532281
532282
532283
532284
532289
532310
532411
532412
532420
532490
533110
541110
541191
541199
541211
541213
541214
541219
541310
541320
541330
541340
541350
541360
541370
541380
541410
541420
541430
541490
541511
541512
541513
541519
541611
541612
541613
541614
541618
541620
541690
541713
541714
541715
541720
541810
541820
541830
541840
541850
541860
541870
541890
541910
541921
541922
541930
541940
541990
551111
551112
551114
561110
561210
561311
561312
561320
561330
561410
561421
561422
561431
561439
561440
561450
561491
561492
561499
561510
561520
561591
561599
561611
561612
561613
561621
561622
561710
561720
561730
561740
561790
561910
561920
561990
562111
562112
562119
562211
562212
562213
562219
562910
562920
562991
562998
611110
611210
611310
611410
611420
611430
611511
611512
611513
611519
611610
611620
611630
611691
611692
611699
611710
621111
621112
621210
621310
621320
621330
621340
621391
621399
621410
621420
621491
621492
621493
621498
621511
621512
621610
621910
621991
621999
622110
622210
622310
623110
623210
623220
623311
623312
623990
624110
624120
624190
624210
624221
624229
624230
624310
624410
711110
711120
711130
711190
711211
711212
711219
711310
711320
711410
711510
712110
712120
712130
712190
713110
713120
713210
713290
713910
713920
713930
713940
713950
713990
721110
721120
721191
721199
721211
721214
721310
722310
722320
722330
722410
722511
722513
722514
722515
811111
811112
811113
811118
811121
811122
811191
811192
811198
811211
811212
811213
811219
811310
811411
811412
811420
811430
811490
812111
812112
812113
812191
812199
812210
812220
812310
812320
812331
812332
812910
812921
812922
812930
812990
813110
813211
813212
813219
813311
813312
813319
813410
813910
813920
813930
813940
813990
814110
921110
921120
921130
921140
921150
921190
922110
922120
922130
922140
922150
922160
922190
923110
923120
923130
923140
924110
924120
925110
925120
926110
926120
926130
926140
926150
927110
928110
928120"""

Six_digit_NAICS = string.split('\n')
print(Six_digit_NAICS)

%store Six_digit_NAICS

['111140', '111150', '111160', '111191', '111199', '111211', '111219', '111310', '111320', '111331', '111332', '111333', '111334', '111335', '111336', '111339', '111411', '111419', '111421', '111422', '111910', '111920', '111930', '111940', '111991', '111992', '111998', '112111', '112112', '112120', '112210', '112310', '112320', '112330', '112340', '112390', '112410', '112420', '112511', '112512', '112519', '112910', '112920', '112930', '112990', '113110', '113210', '113310', '114111', '114112', '114119', '114210', '115111', '115112', '115113', '115114', '115115', '115116', '115210', '115310', '211120', '211130', '212111', '212112', '212113', '212210', '212221', '212222', '212230', '212291', '212299', '212311', '212312', '212313', '212319', '212321', '212322', '212324', '212325', '212391', '212392', '212393', '212399', '213111', '213112', '213113', '213114', '213115', '221111', '221112', '221113', '221114', '221115', '221116', '221117', '221118', '221121', '221122', '221210', '221310',